# Ejercicio c.2 – Incremento de presión en ducto de ventilación forzada

In [57]:
using Pkg
Pkg.add("Unitful")
Pkg.add("Printf")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`



Este notebook resuelve el ejercicio **c.2**, que consiste en determinar el incremento de presión necesario entre la cabina de pintura y la descarga de un sistema de ventilación forzada, para asegurar un caudal de aire de 1500 m³/h en condiciones estándar (15°C y 1 atmósfera). La resolución se realiza con todos los pasos justificados, usando unidades en el SI en el lenguaje de programación científica Julia a través del paquete `Unitful.jl`.


## Interpretación física del enunciado

El ejercicio plantea un sistema de ventilación forzada que extrae aire desde una cabina de pintura hacia el exterior, a través de un conducto horizontal. El objetivo es determinar el **incremento de presión necesario** entre la cabina y la descarga del ventilador para que el **caudal volumétrico estándar** del aire sea de:

$$
Q_{\text{est}} = 1500\,\mathrm{m}^3/\mathrm{h}
$$

correspondiente a condiciones estándar de temperatura y presión:

$$
T_{\text{est}} = 15^\circ\mathrm{C} = 288.15\,\mathrm{K}, \quad P_{\text{est}} = 1\,\mathrm{atm} = 101325\,\mathrm{Pa}
$$

La interpretación de cada uno de los puntos aclaratorios del enunciado es la siguiente:

---

### 🔹 1. Presión y temperatura en la cabina

> *"Considere que la presión atmosférica en la cabina es 1020 hPa al igual que la presión exterior; la temperatura de la cabina es 25°C y en el exterior es 15°C."*

- Se establece que tanto la cabina como el exterior están a **presión atmosférica local**, $P_{\text{cab}} = P_{\text{ext}} = 102000\,\mathrm{Pa}$, aunque a **distinta temperatura**.
- Esta presión es la que se usará para calcular la **densidad real del aire en la cabina** mediante la ley de los gases ideales:

$$
\rho_{\text{cab}} = \frac{P_{\text{cab}}}{R T_{\text{cab}}}
$$

---

### 🔹 2. Despreciar variación de densidad en el ducto

> *"Puede despreciar la variación de densidad del aire en el ducto."*

- Aunque el aire es un gas compresible, se justifica suponer que **\(\rho = \text{cte}\)** en el ducto, ya que las variaciones de presión y temperatura en el tramo considerado son pequeñas.
- Esto permite aplicar el modelo de **flujo incompresible** y usar directamente:
  - ecuación de continuidad,
  - fórmula de Darcy-Weisbach para pérdida de carga distribuida,
  - coeficientes de pérdida localizada.

---

### 🔹 3. Pérdida localizada en la entrada

> *"La constante de pérdida de carga de la entrada puede considerarse igual a 0.5."*

- Se indica que existe una **pérdida localizada** al ingresar el aire al ducto desde la cabina.
- Se modela mediante el coeficiente $K_e = 0.5$, lo cual da lugar a una pérdida de presión:

$$
\Delta p_{\text{entrada}} = \frac{1}{2} \rho v^2 \cdot K_e
$$

- Este valor es típico para entradas sin perfilado, bordes rectos o ingreso brusco.

---

### 🔹 4. Caudal estándar y ley de gases ideales

> *"El caudal en condiciones estándar es el caudal que tendría el flujo másico real si tuviera esta presión y temperatura (es aplicable la Ley de Gases Ideales)."*

- El caudal que se da en el enunciado no es el **caudal real** que circula por el ducto, sino el volumen que ocuparía el flujo másico si estuviera en condiciones estándar.
- Usando la ley de los gases ideales, y asumiendo que la masa es constante, se tiene:




$$
\frac{P_{\text{est}} V_{\text{est}}}{T_{\text{est}}} = \frac{P_{\text{cab}} V_{\text{real}}}{T_{\text{cab}}}
$$

- De aquí se deduce el **caudal real** a condiciones de la cabina:

$$
Q_{\text{real}} = Q_{\text{est}} \cdot \frac{P_{\text{est}}}{P_{\text{cab}}} \cdot \frac{T_{\text{cab}}}{T_{\text{est}}}
$$

Este caudal volumétrico real es el que se usará para calcular la **velocidad media** del aire en el ducto y, con ello, todas las pérdidas de carga.

---

### ✅ Conclusión operativa

- La presión y temperatura reales en la cabina se usan para obtener la densidad del aire.
- Se convierte el caudal estándar a caudal real mediante la ley de gases ideales.
- Se aplica la teoría de pérdida de carga para flujo incompresible, sumando pérdidas localizadas y distribuidas.
- El resultado final será el **incremento de presión mínimo necesario** que el ventilador debe generar para mantener el flujo especificado.

## Paso 1: Datos con unidades en el SI

In [ ]:
using Unitful, Printf

In [69]:

# Dimensiones del ducto
ancho = 0.30u"m"
alto = 0.20u"m"
A = ancho * alto  # área de sección transversal
@show A 

# Longitud y rugosidad
L = 15u"m"
ϵ = 0.15u"mm" |> u"m"  # conversión explícita
@show L
@show ϵ

# Caudal estándar
Q_est_m3h = 1500.0u"m^3/hr"
Q_est = uconvert(u"m^3/s", Q_est_m3h)
@show Q_est

# Presiones y temperaturas
P_est = 101325u"Pa"
T_est = 288.15u"K"
@show P_est
@show T_est

P_cab = 1020u"hPa" |> u"Pa"
T_cab = 298.15u"K"
@show P_cab
@show T_cab

# Pérdida localizada de entrada
Kₑ = 0.5
@show Kₑ

# Constante del gas para aire seco
R = 287.0u"J/(kg*K)"
@show R
#=

Para más adelante jugar con formaos de salida

println(typeof(R))
show ustrip(R)
println(typeof(ustrip(R)))
@show unit(R)
println(typeof(unit(R)))
=#

@printf("R = %.3f %s\n", float(ustrip(R)), string(unit(R)))
# @printf
@printf("Q_est = %.3e %s\n", float(ustrip(Q_est)), string(unit(Q_est)))

A = 0.06 m²
L = 15 m
ϵ = 0.00015 m
Q_est = 0.4166666666666667 m³ s⁻¹
P_est = 101325 Pa
T_est = 288.15 K
P_cab = 102000 Pa
T_cab = 298.15 K
Kₑ = 0.5
R = 287.0 J kg⁻¹ K⁻¹
R = 287.000 J kg⁻¹ K⁻¹
Q_est = 4.167e-01 m³ s⁻¹


## 🔧 Paso 2: Conversión del caudal a condiciones reales

In [60]:

Q_real = Q_est * (P_est / P_cab) * (T_cab / T_est)
@show Q_real;


Q_real = 0.42827368345394734 m³ s⁻¹


## 🔁 Paso 3: Cálculo de la velocidad media en el ducto

In [61]:

v = Q_real / A
@show v;


v = 7.1378947242324555 m s⁻¹


## 📏 Paso 4: Cálculo del diámetro hidráulico

In [62]:

D_h = 2 * ancho * alto / (ancho + alto)
@show D_h;


D_h = 0.24 m


## 💨 Paso 5: Cálculo de la densidad del aire

In [63]:

ρ = P_cab / (R * T_cab)
@show ρ;


ρ = 1.1920197781791433 kg Pa J⁻¹


## 🔄 Paso 6: Cálculo del número de Reynolds

In [64]:

μ = 1.85e-5u"Pa*s"  # viscosidad dinámica a 25°C
Re = ρ * v * D_h / μ
@show Re;


Re = 110380.69214070031 kg m² J⁻¹ s⁻²


## 🌀 Paso 7: Estimación del factor de fricción con Swamee-Jain

In [65]:

ϵ_D = ϵ / D_h
f = 0.25 / (log10(ϵ_D / 3.7 + 5.74 / Re^0.9))^2
@show f;


f = 0.02070332475479568


## 📉 Paso 8: Cálculo de la pérdida de carga total

In [66]:

Δp = 0.5 * ρ * v^2 * (f * L / D_h + Kₑ)
@show Δp;


Δp = 54.47609447494709 kg m² Pa J⁻¹ s⁻²


## ✅ Resultado final

In [67]:

println("\nEl incremento de presión necesario entre cabina y salida del ducto es:")
@show Δp;



El incremento de presión necesario entre cabina y salida del ducto es:
Δp = 54.47609447494709 kg m² Pa J⁻¹ s⁻²
